In [ ]:
import pennylane as qml
from pennylane import numpy as np
# import networkx as nx
from pennylane.optimize import AdamOptimizer
import pandas as pd

In [ ]:
n_assets = 4  # Number of assets
q = 0.5  # Risk aversion parameter
budget = 2

# Vector of expected returns
mu = np.array([0.10, 0.20, 0.15, 0.12])

# Covariance matrix of returns
Sigma = np.array([
    [0.005, -0.010, 0.004, -0.002],
    [-0.010, 0.040, -0.002, 0.004],
    [0.004, -0.002, 0.023, 0.002],
    [-0.002, 0.004, 0.002, 0.018]
])

In [ ]:
# Construct QUBO Hamiltonian
def build_portfolio_hamiltonian(mu, Sigma, q, budget):
    n = len(mu)
    coeffs = []
    observables = []

    # Expected return (maximize => minimize negative)
    for i in range(n):
        coeffs.append(-mu[i])
        observables.append(qml.PauliZ(i))

    # Risk term
    for i in range(n):
        for j in range(i+1, n):
            coeffs.append(q * Sigma[i, j])
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # Budget constraint penalty
    penalty = 30  # change this to tune constraint strength
    for i in range(n):
        for j in range(i+1, n):
            coeffs.append(penalty)
            observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

    return qml.Hamiltonian(coeffs, observables)

H = build_portfolio_hamiltonian(mu, Sigma, q, budget)
print(H)

In [ ]:
# Create VQE circuit
dev = qml.device("default.qubit", wires=n_assets, shots=100)
@qml.qnode(dev)
def circuit(params):
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_assets))
    return qml.expval(H)

In [ ]:
# Optimize the parameters
params = np.random.randn(3, n_assets, 3, requires_grad=True)
opt = AdamOptimizer(stepsize=0.1)

steps = 25
for i in range(steps):
    params = opt.step(circuit, params)
    if i % 10 == 0:
        print(f"Step {i}, Cost = {circuit(params):.4f}")

In [ ]:
# Measure and interpret result
@qml.qnode(dev)
def final_circuit():
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_assets))
    return [qml.sample(qml.PauliZ(i)) for i in range(n_assets)]

result = final_circuit()
portfolio = (1 - np.array(result)) // 2
print("Optimal Portfolio:", portfolio)

In [ ]:
# Convert arrays to strings for grouping
portfolio_strings = [''.join(map(str, row.astype(int))) for row in portfolio]
counts = pd.Series(portfolio_strings).value_counts()

print(counts.head(10))  # Most frequent portfolios

In [ ]:
valid = [sum(row) == budget for row in portfolio]
print(f"Valid portfolios: {sum(valid)}/{len(portfolio)}")